In [23]:
import pandas as pd
import numpy as np
import glob
import unidecode
from collections import defaultdict
import pickle
import editdistance

## 1. Load existing data

In [10]:
data_path = "../../ICLR data/masterdata_unbalanced/"

years = [2017, 2018, 2019, 2020]
papers_data = defaultdict(dict)

for y in years:
    papers_data[y] = pd.read_pickle(data_path + "papers_{}.pkl".format(y))

In [54]:
conflicts_dict = {}
emails_dict = {}

for year in papers_data:
    for paper_id in papers_data[year]:
        year_app_key = str(year) + "_" + paper_id
        
        if "conflicts" in papers_data[year][paper_id]["content"]:
            conflicts_dict[year_app_key] = papers_data[year][paper_id]["content"]["conflicts"]
        else:
            conflicts_dict[year_app_key] = []
            for author_email in papers_data[year][paper_id]["content"]["authorids"]:
                conflicts_dict[year_app_key].append(author_email.split("@")[1])
        
        emails_dict[year_app_key] = papers_data[year][paper_id]["content"]["authorids"]

In [18]:
data_raw = pd.read_pickle('../features/all_data_features_17_20.pkl')
data = data_raw[["id", "title", "label", "authors"]]
org_papers = data.to_dict('index')

In [19]:
data.head(2)

,id,title,label,authors
2017_B1-Hhnslg,2017_B1-Hhnslg,Prototypical Networks for Few-shot Learning,Reject,"[Jake Snell, Kevin Swersky, Richard Zemel]"
2017_B1-q5Pqxl,2017_B1-q5Pqxl,Machine Comprehension Using Match-LSTM and Ans...,Accept,"[Shuohang Wang, Jing Jiang]"


## 2. Start Disambiguation

In [92]:
# per_paper = defaultdict(list)

# still_not_found = 0
# total_auth_count = 0
# found_count = 0
# not_in_gs = 0
# cc = 0

status_track = 0

for p in org_papers:
    status_track += 1
    if status_track % 20 == 0:
        print(status_track)
    if not p in done_papers:
        new_auths = []
        for auth in org_papers[p]["authors"]:
            new_auths += auth.split(" and ")
        
        org_papers[p]["authors"] = org_papers[p]["authors"] + new_auths
        org_papers[p]["authors"] = list(set(org_papers[p]["authors"]))
        
        for auth in org_papers[p]["authors"]:
            if True:
#             try:
                total_auth_count += 1
                disambiguated_author = None

                rm_ast = auth.replace("*", "") 
                unaacented_auth = unidecode.unidecode(rm_ast)
                clean_auth = unaacented_auth.lower()

                key = clean_auth[0]

                all_authors_gs = None

                if ord(key) <= 122 and ord(key) >=97:
                    with open("gs_collation_experiments/author_name_index/{}.pkl".format(key), "rb") as index_files:
                        all_authors_gs = pickle.load(index_files)
                else:
                    print("Non alpha author name: ", auth)
                    all_authors_gs = {}

                if clean_auth in all_authors_gs:
                    if len(all_authors_gs[clean_auth]) == 1:
                        disambiguated_author = all_authors_gs[clean_auth][0][0]
                        found_count += 1
                    else:
                        found_paper = False
                        for mauth in all_authors_gs[clean_auth]:
                            if type(mauth[0]) != dict:
                                fixed_mauth = mauth[0].__dict__
                            else:
                                fixed_mauth = mauth[0]
                            for pub in fixed_mauth["publications"]:
                                if type(pub) != dict:
                                    pub_test = pub.__dict__
                                else:
                                    pub_test = pub
                                org_title = ''.join(filter(str.isalpha, org_papers[p]["title"].lower()))
                                gs_pub_title = ''.join(filter(str.isalpha, pub_test["bib"]["title"].lower()))

                                if org_title == gs_pub_title:
                                    found_paper = True
                                    disambiguated_author = fixed_mauth
                                    found_count += 1
                                    break
                                elif editdistance.eval(org_title, gs_pub_title) < 5:
                                    found_paper = True
                                    disambiguated_author = fixed_mauth
                                    found_count += 1
                                    break
                            if found_paper:
                                break
                        if not found_paper:
                            # Paper not found, check via conflicts/email information
                            matching_affils = []
                            
                            for conflict_affil in conflicts_dict[p]:
                                for mauth in all_authors_gs[clean_auth]:
                                    if type(mauth[0]) != dict:
#                                         print(type(mauth[0]))
#                                         print(dir(mauth[0]))
                                        fixed_mauth = mauth[0].__dict__
                                    else:
                                        fixed_mauth = mauth[0]
                                    if fixed_mauth["email"].find(conflict_affil) > -1:
                                        matching_affils.append(mauth)
#                                         print("FOUND", conflict_affil, fixed_mauth["email"], len(matching_affils))
#                                     else:
#                                         print("NO", conflict_affil, mauth[0]["email"], fixed_mauth["email"], fixed_mauth["email"].find(conflict_affil))
                                    #elif editdistance.eval(conflict_affil, mauth[0].email) < 4:
                                    #    matching_affils.append(mauth)

                            # still not found
                            if len(matching_affils) == 0:
                                still_not_found += 1
                                #print(conflicts_dict[p])
                                #for conflict_affil in conflicts_dict[p]:
                                #    for mauth in all_authors_gs[clean_auth]:
                                #        if mauth[0]["email"].find(conflict_affil) > -1:
                                #            print("FOUND", conflict_affil, mauth[0]["email"], len(matching_affils))
                                #        else:
                                #            print("NO", conflict_affil, mauth[0]["email"])
                            elif len(matching_affils) == 1:
                                disambiguated_author = matching_affils[0][0]
                                found_count += 1
                            else:
                                potential_emailids = []
                                auth_split = clean_auth.split(" ")
                                for splitname in auth_split:
                                    for authemailid in emails_dict[p]:
                                        if authemailid.find(splitname) > -1:
                                            potential_emailids.append(authemailid)

                                potential_emailids = list(set(potential_emailids))
                                if len(potential_emailids) == 1:
                                    domain = potential_emailids[0].split("@")[-1]
                                    identified_auth = []
                                    for m in matching_affils:
                                        if type(m[0]) != dict:
                                            temp_m_somethign = m[0].__dict__
                                        else:
                                            temp_m_somethign = m[0]
                                        
                                        if temp_m_somethign["email"].find(domain) > -1:
                                            identified_auth.append(temp_m_somethign)

                                    if len(identified_auth) == 1:
                                        disambiguated_author = temp_m_somethign
                                        found_count +=1 
                                    else:
                                        still_not_found += 1
                                        #still_not_found_list.append(auth)
                                else:
                                    still_not_found += 1
                                    #still_not_found_list.append(auth)
                    if not disambiguated_author is None:
                        cc +=1
                        per_paper[p].append(disambiguated_author)
                else:
                    not_in_gs +=1
#             except Exception as ex:
#                 print(p, ex)
#                 print(mauth)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940


IndexError: string index out of range

In [93]:
print("Found authors: ", found_count)
print("Added to dict: ", cc)
print("Multiple entries in GS but no publication with the same title found or empty GS: ", still_not_found)
print("NOt in GS data: ", not_in_gs)
print("Total authors to be disambiguated: ", total_auth_count)

Found authors:  8079
Added to dict:  8079
Multiple entries in GS but no publication with the same title found or empty GS:  1558
NOt in GS data:  6164
Total authors to be disambiguated:  15803


In [94]:
for p, v in org_papers.items():
    org_papers[p]["pub_count_gs"] = []
    org_papers[p]["cit_count_gs"] = []
    org_papers[p]["hindex"] = []
    
    # CHECK IF THIS NEEDS TO BE INT
    global_year = int(p.split("_")[0])
    
    for auth_info in per_paper[p]:
        try:
            if type(auth_info) == dict:
                temp_auth_dict = auth_info
            else:
                temp_auth_dict = auth_info.__dict__
            
            if "citedby" in temp_auth_dict and temp_auth_dict["citedby"] > 0:
                total_citations = temp_auth_dict["citedby"]
                if "cites_per_year" in temp_auth_dict:
                    for y in temp_auth_dict["cites_per_year"]:
                        if y > (global_year-1):
                            total_citations -= temp_auth_dict["cites_per_year"][y]
                org_papers[p]["cit_count_gs"].append(total_citations)
            if "publications" in temp_auth_dict:
                total_publications = 0
                for pub in temp_auth_dict["publications"]:
                    if type(pub) != dict:
                        pub_dict = pub.__dict__
                    else:
                        pub_dict = pub
                    if "year" in pub_dict["bib"]:
                        if int(pub_dict["bib"]["year"]) < global_year:
                            total_publications += 1
                    else:
                        total_publications += 1
                org_papers[p]["pub_count_gs"].append(total_publications)
        except Exception as ex:
            print(p,ex)
            break
#     for a in v["authors"]:
#         if a in paper_authors_info:
#             temp_auth_dict = paper_authors_info[a].__dict__
#             if "citedby" in temp_auth_dict and temp_auth_dict["citedby"] > 0:
#                 total_citations = temp_auth_dict["citedby"]
#                 if "cites_per_year" in temp_auth_dict:
#                     for y in temp_auth_dict["cites_per_year"]:
#                         if y > (global_year-1):
#                             total_citations -= temp_auth_dict["cites_per_year"][y]
#                 org_papers[k]["cit_count_gs"].append(total_citations)
#             if "hindex" in temp_auth_dict:
#                 if "hindex5y" in temp_auth_dict:
#                     org_papers[k]["hindex"].append((temp_auth_dict["hindex5y"] + temp_auth_dict["hindex"])/2)
#                 else:
#                     org_papers[k]["hindex"].append(temp_auth_dict["hindex"])
#             if "publications" in temp_auth_dict:
#                 total_publications = 0
#                 for pub in temp_auth_dict["publications"]:
#                     if "year" in pub.__dict__["bib"]:
#                         if pub.__dict__["bib"]["year"] < global_year:
#                             total_publications += 1
#                     else:
#                         total_publications += 1
#                 org_papers[k]["pub_count_gs"].append(total_publications)

In [95]:
done_papers = []

for k, v in org_papers.items():
    if v["pub_count_gs"]:
        #print(k)
        done_papers.append(k)

done_papers = list(set(done_papers))

In [47]:
for i in per_paper["2018_BkSDMA36Z"]:
    print(i.__dict__["citedby"])
    print(type(i))

3294
<class 'scholarly.scholarly.Author'>


In [49]:
for i in per_paper["2020_rJeW1yHYwH"]:
#     print(i.__dict__["citedby"])
    print(type(i))

<class 'dict'>


In [37]:
for i in auth_info["publications"]:
    print(i["bib"])

{'cites': '10', 'year': '2018', 'title': 'Deep multi-scale architectures for monocular depth estimation'}
{'cites': '0', 'year': '2019', 'title': 'n-MeRCI: A new Metric to Evaluate the Correlation Between Predictive Uncertainty and True Error'}
{'cites': '0', 'year': '2019', 'title': "Estimation de profondeur à partir d'images monoculaires par apprentissage profond"}


In [96]:
with open("runnnnn.pkl", "wb") as run:
    pickle.dump(org_papers, run)

In [52]:
list(org_papers.items())[0:3]

[('2019_SJMnG2C9YX',
  {'authors': ['Aditya Krishna Menon',
    'Takashi Ishida',
    'Masashi Sugiyama',
    'Gang Niu'],
   'cit_count_gs': [],
   'hindex': [],
   'id': '2019_SJMnG2C9YX',
   'label': 'Reject',
   'pub_count_gs': [],
   'title': 'Complementary-label learning for arbitrary losses and models'}),
 ('2020_B1erJJrYPH',
  {'authors': ['Payel Das',
    'Prasanna Sattigeri',
    'N. Joseph Tatro',
    'Pin-Yu Chen',
    'Rongjie Lai',
    'Igor Melnyk'],
   'cit_count_gs': [],
   'hindex': [],
   'id': '2020_B1erJJrYPH',
   'label': 'Reject',
   'pub_count_gs': [],
   'title': 'Optimizing Loss Landscape Connectivity via Neuron Alignment'}),
 ('2019_Byxpfh0cFm',
  {'authors': ['Virginia Smith', 'Michael Kuchnik'],
   'cit_count_gs': [],
   'hindex': [],
   'id': '2019_Byxpfh0cFm',
   'label': 'Accept',
   'pub_count_gs': [],
   'title': 'Efficient Augmentation via Data Subsampling'})]